# Final Project (Part 4)
#### By Shubhankar Kumar

### 5. Technology associated with rapidly growing repositories

In [1]:
commits_df = spark.read.parquet("gs://msca-bdp-students-bucket/shared_data/shubhankark/commits")

In [2]:
languages_df = spark.read.parquet("gs://msca-bdp-data-open/final_project_git/languages")

In [28]:
languages_df.show(5)

+-------------------+------------+
|          repo_name|    language|
+-------------------+------------+
|  lemi136/puntovent|   [{C, 80}]|
|     taxigps/nctool| [{C, 4461}]|
|        ahy1/strbuf| [{C, 5573}]|
|nleiten/mod_rpaf-ng|[{C, 30330}]|
|kmcallister/alameda|[{C, 17077}]|
+-------------------+------------+
only showing top 5 rows



In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, sum, dense_rank
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import seaborn as sns

# Specify the repo_names
target_repo_names = ["shenzhouzd/update", "duaneking/rockstar_test", "Dark5ide/mycroft-core"]

# Filter languages_df for the specified repo_names
filtered_languages_df = languages_df.filter(col("repo_name").isin(target_repo_names))

In [35]:
exploded_languages_df = filtered_languages_df.select("repo_name", explode("language").alias("lang"))

# Group by repo_name and language name, then sum the bytes for each group
result_df = exploded_languages_df.groupBy("repo_name", col("lang.name").alias("language_name")).agg(sum("lang.bytes").alias("byte_count")).orderBy("repo_name", col("byte_count").desc())

# Rank the languages based on byte count within each repo_name
windowSpec = Window.partitionBy("repo_name").orderBy(col("byte_count").desc())
result_df = result_df.withColumn("rank", dense_rank().over(windowSpec))

In [36]:
top3_languages_df = result_df.filter(col("rank") <= 3)

# Convert to Pandas for plotting
top3_languages_pd = top3_languages_df.toPandas()

In [39]:
top3_languages_pd

,repo_name,language_name,byte_count,rank
0,duaneking/rockstar_test,COBOL,130,1
1,duaneking/rockstar_test,Brainfuck,111,2
2,duaneking/rockstar_test,Pascal,109,3
3,Dark5ide/mycroft-core,Python,1846776,1
4,Dark5ide/mycroft-core,Shell,80311,2
5,Dark5ide/mycroft-core,QML,9903,3
6,shenzhouzd/update,JavaScript,79741,1
7,shenzhouzd/update,HTML,8574,2


In [40]:
licenses_df = spark.read.parquet("gs://msca-bdp-data-open/final_project_git/licenses")

In [41]:
licenses_df.show(5)

+--------------------+------------+
|           repo_name|     license|
+--------------------+------------+
|autarch/Dist-Zill...|artistic-2.0|
|thundergnat/Prime...|artistic-2.0|
|kusha-b-k/Turabia...|artistic-2.0|
|onlinepremiumoutl...|artistic-2.0|
|huangyuanlove/Lia...|artistic-2.0|
+--------------------+------------+
only showing top 5 rows



In [42]:
target_repo_names = ["shenzhouzd/update", "duaneking/rockstar_test", "Dark5ide/mycroft-core"]

# Filter licenses_df for the specified repo_names
filtered_licenses_df = licenses_df.filter(col("repo_name").isin(target_repo_names))

# Rank the licenses based on count within each repo_name
windowSpec = Window.partitionBy("repo_name").orderBy(col("count").desc())
ranked_licenses_df = filtered_licenses_df.groupBy("repo_name", "license").count().withColumn("rank", dense_rank().over(windowSpec))

# Filter for the top 3 licenses in each repo_name
top3_licenses_df = ranked_licenses_df.filter(col("rank") <= 3)

# Convert to Pandas for plotting
top3_licenses_pd = top3_licenses_df.toPandas()

In [43]:
top3_licenses_pd

,repo_name,license,count,rank
0,duaneking/rockstar_test,agpl-3.0,1,1
1,Dark5ide/mycroft-core,apache-2.0,1,1
2,shenzhouzd/update,agpl-3.0,1,1
